In [2]:
from datasets import load_dataset

data = load_dataset("wikipedia", "2ƒ0220301.simple", split='train[:10000]')
data

Downloading: 100%|██████████| 1.66k/1.66k [00:00<00:00, 1.66MB/s]
Downloading: 100%|██████████| 235M/235M [00:47<00:00, 4.92MB/s] 


Dataset wikipedia downloaded and prepared to r:/HuggingFace/Cache/Datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 10000
})

In [3]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
data = load_dataset("wikipedia", "20220301.simple", split='train[:10000]')

# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(data)

df = df.drop(columns = ["text"])
# Dump the first 10 records
print(df.head(10))

Found cached dataset wikipedia (r:/HuggingFace/Cache/Datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


   id                                                url  \
0   1            https://simple.wikipedia.org/wiki/April   
1   2           https://simple.wikipedia.org/wiki/August   
2   6              https://simple.wikipedia.org/wiki/Art   
3   8                https://simple.wikipedia.org/wiki/A   
4   9              https://simple.wikipedia.org/wiki/Air   
5  12  https://simple.wikipedia.org/wiki/Autonomous%2...   
6  13    https://simple.wikipedia.org/wiki/Alan%20Turing   
7  14  https://simple.wikipedia.org/wiki/Alanis%20Mor...   
8  17  https://simple.wikipedia.org/wiki/Adobe%20Illu...   
9  18        https://simple.wikipedia.org/wiki/Andouille   

                             title  
0                            April  
1                           August  
2                              Art  
3                                A  
4                              Air  
5  Autonomous communities of Spain  
6                      Alan Turing  
7                Alanis Morissette  
8     

In [4]:
data[6]

{'id': '13',
 'url': 'https://simple.wikipedia.org/wiki/Alan%20Turing',
 'title': 'Alan Turing',
 'text': 'Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dub

Create the tokenizer that helps to understand how many token is a text.

In [5]:
import tiktoken  # !pip install tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")

28

We have nice components that can split text for us.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)


In [7]:
from pprint import pprint
chunks = text_splitter.split_text(data[6]['text'])
pprint(chunks[:2])

# cycle in all chunks and print length with previous function
for chunk in chunks:
    print(tiktoken_len(chunk))

['Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 '
 'June 1954) was an English mathematician and computer scientist. He was born '
 'in Maida Vale, London.\n'
 '\n'
 'Early life and family \n'
 'Alan Turing was born in Maida Vale, London on 23 June 1912. His father was '
 'part of a family of merchants from Scotland. His mother, Ethel Sara, was the '
 'daughter of an engineer.\n'
 '\n'
 'Education \n'
 "Turing went to St. Michael's, a school at 20 Charles Road, St "
 'Leonards-on-sea, when he was five years old.\n'
 '"This is only a foretaste of what is to come, and only the shadow of what is '
 'going to be.” – Alan Turing.\n'
 '\n'
 'The Stoney family were once prominent landlords, here in North Tipperary. '
 'His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller '
 'Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. '
 'Longford); Protestant Anglo-Irish gentry.\n'
 '\n'
 'Educated in Dublin at Alexandra School an

Now we need to use embeddings, remmeber we need to use code for azure.

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv, find_dotenv
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_type = 'azure'
openai.api_version = '2023-05-15'

_ = load_dotenv(find_dotenv()) # read local .env file

model_name = 'text-embedding-ada-002'
api_key = os.getenv("OPENAI_API_KEY")

# even if the api is azure, we can simply set as above the base class to work with azure
# then use standard embeddings
embed = OpenAIEmbeddings(
    openai_api_key=api_key
)



In [10]:
texts = [
    'this is the first chunk of text',
]

res = embed.embed_documents(texts)
len(res), len(res[0])

(1, 1536)

Now integrate with pinecone

In [11]:
import pinecone

_ = load_dotenv(find_dotenv()) # read local .env file

index_name = 'langchain-retrieval-augmentation'

pinecone.init(
        api_key=os.getenv("PINECONE_KEY"),  # find api key in console at app.pinecone.io
        environment=os.getenv("PINECONE_ENV")  # find next to api key in console
)

# print pinecone key and env
# print(os.getenv("PINECONE_KEY"))
print(os.getenv("PINECONE_ENV"))


us-west1-gcp-free


In [12]:

# we create a new index
pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=len(res[0]) # 1536 dim of text-embedding-ada-002
)


In [11]:
import pinecone
grpcindex = pinecone.GRPCIndex(index_name)

grpcindex.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [18]:
from tqdm.auto import tqdm
from uuid import uuid4

# switch to normal index forindexing, not using GRPC indexes because it will throws error
index = pinecone.Index(index_name=index_name)
batch_limit = 50

texts = []
metadatas = []

for i, record in enumerate(tqdm(data)):
    # first get metadata fields for this record
    metadata = {
        'wiki-id': str(record['id']),
        'source': record['url'],
        'title': record['title']
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['text'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)

    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        # ok azure api cannot embed more than one text at a time we need to call each
        # chunk of text separately for each text in texts
        embeds = [embed.embed_documents([text]) for text in texts]
        
        # print length of texts ids and embeds verify that they are all the same
        # print(len(texts), len(ids), len(embeds), len(metadatas))

        # in OpenAI embeddings we can embed all texts at once
        # embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []


 32%|███▏      | 3205/10000 [34:23<1:12:55,  1.55it/s]


KeyboardInterrupt: 

In [19]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)


In [20]:
query = "who was Benito Mussolini?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='Benito Amilcare Andrea Mussolini KSMOM GCTE (29 July 1883 – 28 April 1945) was an Italian politician and journalist. He was also the Prime Minister of Italy from 1922 until 1943. He was the leader of the National Fascist Party.\n\nBiography\n\nEarly life\nBenito Mussolini was named after Benito Juarez, a Mexican opponent of the political power of the Roman Catholic Church, by his anticlerical (a person who opposes the political interference of the Roman Catholic Church in secular affairs) father. Mussolini\'s father was a blacksmith. Before being involved in politics, Mussolini was a newspaper editor (where he learned all his propaganda skills) and elementary school teacher.\n\nAt first, Mussolini was a socialist, but when he wanted Italy to join the First World War, he was thrown out of the socialist party. He \'invented\' a new ideology, Fascism, much out of Nationalist\xa0and Conservative views.\n\nRise to power and becoming dictator\nIn 1922, he took power b

In [23]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# completion llm
llm = AzureChatOpenAI(
	temperature=0,
	openai_api_version="2023-03-15-preview",
    deployment_name="gpt35", 
	model_name="gpt-35-turbo"
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


In [25]:
pprint(data[10])

{'id': '19',
 'text': 'Farming is growing crops and keeping animals  for food and raw '
         'materials. Farming is a part of agriculture. \n'
         '\n'
         'Agriculture started thousands of years ago, but no one knows for '
         'sure how old it is. The development of farming gave rise to the '
         'Neolithic Revolution as people gave up nomadic hunting and became '
         'settlers in cities. \n'
         '\n'
         'Agriculture and domestication probably started in the Fertile '
         'Crescent (the Nile Valley, the Levant and Mesopotamia). The area '
         'called Fertile Crescent is now in the countries of Iraq, Syria, '
         'Turkey, Jordan, Lebanon, Israel, and Egypt. Wheat and barley are '
         'some of the first crops people grew. People probably started '
         'agriculture slowly by planting a few crops, but still gathered many '
         'foods from the wild. People may have started farming because the '
         'weather and soil

In [26]:
qa.run("which is the most useful suggestion for modern agriculture?")

'There are several suggestions for modern agriculture mentioned in the context, but it is difficult to determine which one is the most useful as it depends on the specific situation. Some suggestions include selecting plants with better yield, taste, and nutritional value, using genetic engineering to improve crops, using natural products in organic farming, and using local resources in integrated farming. Additionally, modern young farmers usually have a good technical education to help solve problems related to pollution, erosion, diseases, pests, weeds, drought, rainfall, climate, and contamination.'

In [27]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [29]:
qa_with_sources("which is the most useful suggestion for modern agriculture?")

{'question': 'which is the most useful suggestion for modern agriculture?',
 'answer': 'The most useful suggestion for modern agriculture is to select plants with better yield, taste, and nutritional value, as well as those that can survive plant disease and drought, and are easier to harvest. Additionally, using genetic engineering to improve plants and implementing organic or integrated farming practices can also be beneficial. However, it is important to address the problems of pollution, erosion, diseases, pests, weeds, drought, rainfall, climate, and contamination that can arise in agriculture. \n',
 'sources': 'https://simple.wikipedia.org/wiki/Farming'}